In [2]:
from glob import glob
import os
import pandas as pd

WORK_DIR = "/bheuts/ANANSE-CAGE/data/hg19"

In [ ]:
# Optional
# Motif scan to speed up ANANSE benchmarking
for dir_name in os.listdir(f"{WORK_DIR}"):
    if not dir_name == "220504_ANANSE-CAGE-run.ipynb" and not dir_name == ".ipynb_checkpoints":
        for fname in glob(f"{dir_name}/enhancers_*.bed"): 
            ! rm {fname}
        for fname in glob(f"{dir_name}/enhancers_*.tsv"): 
            fname_new = fname.replace(".tsv", ".bed")
            df = pd.read_table(fname)
            regions = df["pos"].str.split('[:-]', expand=True)
            regions.columns = ["chrom", "start", "end"]
            regions["start"] = regions["start"].astype(int)
            regions["end"] = regions["end"].astype(int)

            center = ((regions["start"] + regions["end"]) / 2).astype(int)
            regions["start"] = center - 100
            regions["end"] = center + 100

            regions.to_csv(fname_new, header=False, index=False, sep="\t")

        for fname in glob(f"{dir_name}/enhancers_*.bed"):         
            fname_new = fname.replace(".bed", "")
            ! nice -n 15 gimme scan -Tz --gc -g hg19 {fname} > {fname_new}_SCAN.tsv

In [ ]:
EDGES = [
    500000
]
JACCARD_CUTOFF = [
    0.1,
]

OUT_DIR = "results"


# ANANSE RUN
for dir_name in os.listdir(f"{WORK_DIR}"):
    if not dir_name == "220504_ANANSE-CAGE-run.ipynb" and not dir_name == ".ipynb_checkpoints":
        
        for cutoff in JACCARD_CUTOFF:
            if cutoff == 0.1:
                OUT_DIR_JACCARD = f"{OUT_DIR}/jaccard_01"
            if not os.path.exists(f"{dir_name}/{OUT_DIR_JACCARD}"):
             
                # ANANSE BINDING
                # Source
                ! nice -n 15 ananse binding -C {dir_name}/enhancers_source.tsv -o {dir_name}/{OUT_DIR_JACCARD}/source_binding -g hg19 --jaccard-cutoff {cutoff} --pfmscorefile {dir_name}/enhancers_source_SCAN.tsv
                
                # Target
                ! nice -n 15 ananse binding -C {dir_name}/enhancers_target.tsv -o {dir_name}/{OUT_DIR_JACCARD}/target_binding -g hg19 --jaccard-cutoff {cutoff} --pfmscorefile {dir_name}/enhancers_target_SCAN.tsv


                # ANANSE NETWORK
                # Source
                ! nice -n 15 ananse network {dir_name}/{OUT_DIR_JACCARD}/source_binding/binding.h5 -e {dir_name}/TPM_source_*.txt -o {dir_name}/{OUT_DIR_JACCARD}/network_source.tsv -g hg19
                
                # Target
                ! nice -n 15 ananse network {dir_name}/{OUT_DIR_JACCARD}/target_binding/binding.h5 -e {dir_name}/TPM_target_*.txt -o {dir_name}/{OUT_DIR_JACCARD}/network_target.tsv -g hg19

            for edge in EDGES:
                if edge == 500000:
                    OUT_DIR_EDGE = f"{OUT_DIR_JACCARD}/edges_500k"

                if not os.path.exists(f"{dir_name}/{OUT_DIR_EDGE}"):
                    ! mkdir {dir_name}/{OUT_DIR_EDGE}

            # ANANSE INFLUENCE
            ! nice -n 15 ananse influence -s {dir_name}/{OUT_DIR_JACCARD}/network_source.tsv -t {dir_name}/{OUT_DIR_JACCARD}/network_target.tsv -d {dir_name}/DE_genes.txt -o {dir_name}/{OUT_DIR_EDGE}/influence.tsv -i {edge} -n 24
            ! nice -n 15 ananse plot {dir_name}/{OUT_DIR_EDGE}/influence.tsv -d {dir_name}/{OUT_DIR_EDGE}/influence_diffnetwork.tsv -o {dir_name}/{OUT_DIR_EDGE}